### RAG Application Using Type Sense

In [1]:
import typesense

In [ ]:
client=typesense.Client({
  'nodes': [{
    'host': 'yf2l38ewmgkuc1isp-1.a1.typesense.net',  # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',       # For Typesense Cloud use 443
    'protocol': 'https'    # For Typesense Cloud use https
  }],
  'api_key':'80SHa794fhdmVH0kog60bZhLMgUFhPpQ',
  'connection_timeout_seconds': 10
})

books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))

{'created_at': 1762754734, 'default_sorting_field': 'ratings_count', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'title', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'authors', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string[]'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'publication_year', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'ratings_count', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'average_rating', 'optional': False, 'sort': True, 'stem': Fal

In [6]:
client

In [9]:
batch = []
batch_size = 100

with open('books.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        batch.append(line)
        if len(batch) == batch_size:
            client.collections['books'].documents.import_("\n".join(batch))
            batch = []

    if batch:
        client.collections['books'].documents.import_("\n".join(batch))


In [10]:
search_parameters={
    'q':"harry potter",
    'query_by':"title,authors",
    'sort_by':"ratings_count:desc"
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 17,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}},
  {'document': {'authors': ['J.K. Rowling', ' Mary GrandPré', ' R

In [11]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [12]:
search_parameters = {
  'q'         : 'experyment',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [{'counts': [{'count': 1,
     'highlighted': ' Käthe Mazur',
     'value': ' Käthe Mazur'},
    {'count': 1, 'highlighted': 'Mahatma Gandhi', 'value': 'Mahatma Gandhi'},
    {'count': 1, 'highlighted': 'Gretchen Rubin', 'value': 'Gretchen Rubin'},
    {'count': 1,
     'highlighted': 'James Patterson',
     'value': 'James Patterson'}],
   'field_name': 'authors',
   'sampled': False,
   'stats': {'total_values': 4}}],
 'found': 3,
 'hits': [{'document': {'authors': ['James Patterson'],
    'average_rating': 4.08,
    'id': '569',
    'image_url': 'https://images.gr-assets.com/books/1339277875m/13152.jpg',
    'publication_year': 2005,
    'ratings_count': 172302,
    'title': 'The Angel Experiment'},
   'highlight': {'title': {'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}],
   'text_match': 5787300

### Langchain + Typsense + Groq LLM + RAG Application

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [16]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [17]:
loader = TextLoader("test.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

C:\Users\Rakshith.kumar\AppData\Local\Temp\ipykernel_27356\13807947.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\Rakshith.kumar\AppData\Local\Temp\ipykernel_27356\13807947.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\Rakshith.kumar\Desktop\AIDEMO\AIDEMO\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficient